## Introduction

This notebook demonstrates selecting a trajectory according to its ID 
and explore map matching results in notebook with map and diagrams. 

### Read GPS file into a dataframe

In [1]:
traj_csv_file = "data/stockholm/traj.csv"
gps_csv_file = "data/stockholm/gps.csv"

In [2]:
def geojson2wkt(geojson):
    if "geometry" in geojson:
        return shape(geojson["geometry"]).wkt
    return ""
def traj2gps(traj_df):
    return gps_df

In [3]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

### Read GPS trajectory data into a data frame

In [4]:
df = pd.read_csv(traj_csv_file,sep=";")
df['geom'] = df['geom'].apply(wkt.loads)

In [5]:
gdf = gpd.GeoDataFrame(df, geometry = 'geom')

### Convert GPS trajectory dataframe into a GPS dataframe

In [28]:
gps_data = []
for index,row in gdf.iterrows():
    print row.id, row.geom
    for x,y in row.geom.coords:
        gps_data.append((row.id,x,y))
gps_df = pd.DataFrame(gps_data,columns=["id","x","y"])
gps_df.to_csv(gps_csv_file,sep=";",index=False)

1 LINESTRING (18.072906 59.336914, 18.077199 59.342341, 18.0687 59.34733, 18.058913 59.350787, 18.053677 59.351224, 18.053419 59.346586, 18.049728 59.342472, 18.046551 59.341991)
2 LINESTRING (18.030938 59.343909, 18.041072 59.335943, 18.048851 59.341621, 18.05128 59.346123, 18.069409 59.341881)
3 LINESTRING (18.072994 59.338227, 18.069045 59.336826, 18.062177 59.336739, 18.056683 59.341334, 18.054107 59.343742, 18.054107 59.346761, 18.050759 59.349387, 18.045523 59.350568, 18.040286 59.348993, 18.0402 59.345098)
    id          x          y
0    1  18.072906  59.336914
1    1  18.077199  59.342341
2    1  18.068700  59.347330
3    1  18.058913  59.350787
4    1  18.053677  59.351224
5    1  18.053419  59.346586
6    1  18.049728  59.342472
7    1  18.046551  59.341991
8    2  18.030938  59.343909
9    2  18.041072  59.335943
10   2  18.048851  59.341621
11   2  18.051280  59.346123
12   2  18.069409  59.341881
13   3  18.072994  59.338227
14   3  18.069045  59.336826
15   3  18.062177

### Visualize the result

In [35]:
gdf.head()

,id,geom
0,1,"LINESTRING (18.072906 59.336914, 18.077199 59...."
1,2,"LINESTRING (18.030938 59.343909, 18.041072 59...."
2,3,"LINESTRING (18.072994 59.338227, 18.069045 59...."


In [92]:
class TrajSelector:
    def __init__(self, data, id_column="id", geom_column="geom"):
        from ipywidgets import Dropdown,Layout
        self.data = data
        self.id_column = id_column
        self.geom_column = geom_column
        self.current_id = data.loc[0,id_column]
        self.dropdown = Dropdown(
            options=data[id_column].values,
            value= self.current_id,
            description='Traj id:',
            layout = Layout(width="160px"),
            style = {'description_width': '30%'}
        )   
        self.dropdown.observe(self.__observe, names=['value'])
        self.__callback = None
    def get_current_traj_id(self):
        return self.dropdown.value
    def get_current_traj_geom(self):
        return self.data[self.data[self.id_column]==self.dropdown.value][self.geom_column].values[0]
    def __observe(self, change):
        if self.__callback!=None:
            self.__callback(self.get_current_traj_id(), self.get_current_traj_geom())
    def register_listener(self, callback):
        self.__callback = callback

In [93]:
traj_selector = TrajSelector(df)

def callback_v2(tid, value):
    print(tid, value.wkt)

traj_selector.register_listener(callback_v2)
traj_selector.dropdown

RHJvcGRvd24oZGVzY3JpcHRpb249dSdUcmFqIGlkOicsIGxheW91dD1MYXlvdXQod2lkdGg9dScxNjBweCcpLCBvcHRpb25zPSgxLCAyLCAzKSwgc3R5bGU9RGVzY3JpcHRpb25TdHlsZShkZXPigKY=


(2, 'LINESTRING (18.030938 59.343909, 18.041072 59.335943, 18.048851 59.341621, 18.05128 59.346123, 18.069409 59.341881)')
(3, 'LINESTRING (18.072994 59.338227, 18.069045 59.336826, 18.062177 59.336739, 18.056683 59.341334, 18.054107 59.343742, 18.054107 59.346761, 18.050759 59.349387, 18.045523 59.350568, 18.040286 59.348993, 18.0402 59.345098)')


In [94]:
class MMConfigSelector:
    def __init__(self):
        from ipywidgets import IntSlider, Layout
        self.k_slider = IntSlider(description='k:', min=4, max=32, value=20,step=4, 
                     layout= Layout(width="160px"), style = {'description_width': '10%'})
        self.r_slider = IntSlider(description='r:', min=50, max=500, value=300, step =50,
                            layout= Layout(width="160px"),style = {'description_width': '10%'})
        self.e_slider = IntSlider(description='e:', min=50, max=200, value=140, step = 30,
                            layout= Layout(width="160px"), style = {'description_width': '10%'})
        self.k_slider.observe(self.__observe,names=['value'])
        self.r_slider.observe(self.__observe,names=['value'])
        self.e_slider.observe(self.__observe,names=['value'])
        self.__callback = None
    def get_config(self):
        return (self.k_slider.value, self.r_slider.value, self.e_slider.value)
    def __observe(self, change):
        if self.__callback!=None:
            self.__callback(self.get_config())
    def register_listener(self, callback):
        self.__callback = callback

In [95]:
mm_config = MMConfigSelector()

In [96]:
def callback(value):
    print(value)
mm_config.register_listener(callback)

In [97]:
import pandas as pd

In [100]:
ids = [1,2,3,4,5]
a = [5,6,7,8,9]
b = [1.1,2,3.5,4,6]
c = [6,7,8,4,3]
df = pd.DataFrame([],columns=["id","a","b","c"])
df.id = ids
df.a = a
df.b = b
df.c = c

In [101]:
df.head()

,id,a,b,c
0,1,5,1.1,6
1,2,6,2.0,7
2,3,7,3.5,8
3,4,8,4.0,4
4,5,9,6.0,3


In [102]:
df.shape

(5, 4)

In [104]:
df.columns.values

array(['id', 'a', 'b', 'c'], dtype=object)

### DataFrameVisualizer

In [ ]:
class DataFrameVisualizer:
    ATTRIBUTES = ["error","length","offset","spdist","ep","tp"]
    DESCRIPTIONS = ["GPS error (meter)","Edge length (meter)",
                        "Offset (meter)","SP length (meter)",
                        "Emission Proability","Transit Probability"]
    DEFAULT_ATTRIBUTE = "error"
    def __init__(self, df):
        import numpy as np
        from bqplot import LinearScale, Axis, Figure, ColorScale, Scatter
        self.df = df
        self.sc_x = LinearScale()
        self.sc_y = LinearScale()
        self.value_label = Label()
        self.N = df.shape[0]
        self.X = np.arange(N)
        dropdown_options = []
        for d,v in zip(DESCRIPTIONS,ATTRIBUTES):
            dropdown_options.append((d,v))
        self.dropdown = Dropdown(
            options=dropdown_options,
            value=DEFAULT_ATTRIBUTE,
            description='Feature:',
            layout=Layout(
                width="50%"   
            )
        )
        self.selected_field = DEFAULT_ATTRIBUTE
        self.ax_x = Axis(scale=self.sc_x, label='Point index')
        self.ax_y = Axis(scale=self.sc_y, orientation='vertical', label='-')
        self.fig = Figure(marks=[self.plot], axes=[self.ax_x, self.ax_y])
        self.dropdown.observe(self.__dropdown_observe, names=['value'])
        self.plot = Scatter(x=self.X, y=self.df[self.selected_field].values, scales={'x': self.sc_x, 'y': self.sc_y},
            default_size=200,colors=["orange"],hovered_style={             
            "fill":"white",
            "stroke":"#9932CC",
            "stroke-width":5
        })
        self.__field_change_callback = None
        self.__hover_callback = None
        self.__unhover_point_callback = None
        self.plot.on_hover(__hover)
        self.plot.observe(__observe)
        self.chart = VBox([self.value_label, self.fig])
    def __dropdown_observe(self, change):
        if self.__field_change_callback!=None:
            self.__field_change_callback(get_selected_field(),
                                         self.df[get_selected_field()].values)
    def __hover(self, target):
        self.hovered_id = target["data"]["index"]
        self.label.value = "{.4:f}".format(self.df[self.selected_field].values[self.hovered_id])
        if self.__hover_callback!=None:
            self.__hover_callback(self.hovered_id)
    def __unhover(self, change):
        if self.__unhover_callback!=None:            
            self.__unhover_callback(get_selected_field(),
                                  self.df[get_selected_field()].values,
                                  index)        
    def __observe(self, state):
        if (self.plot.hovered_point==None):
            self.label.value = ""
            if (self.hover_state):
                self.hover_state = False            
                self.__unhover(self.)
    def get_selected_field(self):
        return self.dropdown.value
    def select_field(self, field):
        self.dropdown.value = field
        self.plot.y = self.data[field]
        self.selected_field = field
    def reset_data(self,df):
        self.df = df
        self.__reset_data_plot()
    def __reset_data_plot(self):
        self.plot.x = np.arange(self.df.shape[0])
        self.plot.y = self.df[self.selected_field].values
    def on_field_change(self,callback):
        self.__field_change_callback = callback
    def on_hovered_point(self, callback):
        self.__hover_callback = callback
    def on_unhovered_point(self, callback):
        self.__unhover_point_callback = callback
    def set_hover_point(self, index):
        
    def get_hovered_point(self, index):
        

### TrajMap

In [ ]:
class TrajMap:
    def __init__(df, k_range, r_range, e_range):
        self.map
        self.traj_layer
        self.mr_layer
        self.label
        self.hovered_feature =
    def __observe(self):
        
    def __hover(self):
        return None
    def on_hover(self):
        return None
    def __unhover(self):
        
    def un_hover(self);
        

### Define the application

In [ ]:
class DFMMExplorer():
    def __init__(self, df):
        self.map = TrajMap()
        self.df_viewer = DataFrameViewer()
        self.traj_selector = TrajSelector()
        self.ui = HBox()
    def show():
        return self.ui
    def set_data(self,df): 

In [29]:
from ipyleaflet import Map, basemaps, WidgetControl, DrawControl
from ipywidgets import IntSlider, ColorPicker, jslink, FloatSlider, Button, Layout, Label, Dropdown,HBox,VBox

# Add widget
ml = Layout(flex="l",height="540px",width="50%")
m = Map(center=(59.341884644077, 18.06016188114882), zoom=14, 
        basemap=basemaps.OpenStreetMap.Mapnik,layout=ml)


k_slider = IntSlider(description='k:', min=4, max=32, value=20,step=4, 
                     layout= control_layout, style = {'description_width': '10%'})
k_widget_control = WidgetControl(widget=k_slider, position='bottomright')

r_slider = IntSlider(description='r:', min=50, max=500, value=300, step =50,
                    layout= control_layout, style = {'description_width': '10%'})
r_widget_control = WidgetControl(widget=r_slider, position='bottomright')

e_slider = IntSlider(description='e:', min=50, max=200, value=140, step = 30,
                    layout= control_layout, style = {'description_width': '10%'})
e_widget_control = WidgetControl(widget=e_slider, position='bottomright')

m.add_control(e_widget_control)
m.add_control(r_widget_control)
m.add_control(k_widget_control)

label_widget= Label(layout = control_layout)
label_widget.value = "Match status: {} ".format("  -  ") 
label_control = WidgetControl(widget=label_widget, position='topright',max_width = 320)
m.add_control(label_control)

### Add event listensers

mr_layer = None
traj_layer = None

def clear_map():
    global traj_layer, mr_layer
    if (mr_layer!=None):
        m.remove_layer(mr_layer) 
        mr_layer = None
    if (traj_layer!=None):
        m.remove_layer(traj_layer) 
        traj_layer = None
    label_widget.value = "Match status: {} ".format("-")     

def update_map(data):
    global traj_layer, mr_layer
    if data==None:
        if traj_layer!=None:
            m.remove_layer(traj_layer)
            if (mr_layer!=None):
                m.remove_layer(mr_layer)
                mr_layer = None
            temp_mr_layer = match_geojson_network(traj_layer.data,
                k_slider.value,r_slider.value,e_slider.value)
            if (temp_mr_layer!=None):
                label_widget.value = "Match status: {} ".format("True")
                m.add_layer(temp_mr_layer)
                mr_layer = temp_mr_layer
            else:
                label_widget.value = "Match status: {} ".format("False") 
            m.add_layer(traj_layer)
        else:
            label_widget.value = "Match status: {} ".format("-") 
    else:
        temp_mr_layer = match_geojson_network(data,
            k_slider.value,r_slider.value,e_slider.value)
        if temp_mr_layer!=None:
            label_widget.value = "Match status: {} ".format("True") 
            if (mr_layer!=None):
                m.remove_layer(mr_layer)
                mr_layer = None
            temp_traj_layer = traj_geojson_layer(data)    
            if (traj_layer!=None):
                m.remove_layer(traj_layer)
                traj_layer=None
            m.add_layer(temp_mr_layer)
            m.add_layer(temp_traj_layer)
            mr_layer = temp_mr_layer
            traj_layer = temp_traj_layer
        else:
            label_widget.value = "Match status: {} ".format("False") 
            temp_traj_layer = traj_geojson_layer(data)    
            if (traj_layer!=None):
                m.remove_layer(traj_layer)
                traj_layer = None
            m.add_layer(temp_traj_layer)
            traj_layer = temp_traj_layer

def on_value_change(change):
    update_map(None)
    


### Define the details visualizer 

In [30]:
from bqplot import LinearScale, Lines, Axis, Figure, ColorScale, Scatter

In [33]:
import numpy as np
N = 10
# N = details.shape[0]
X = np.arange(N)
sc_x = LinearScale()
sc_y = LinearScale()

attributes = ["error","length","offset","spdist","ep","tp"]
descriptions = ["GPS error (meter)","Edge length (meter)",
                "Offset (meter)","SP length (meter)",
                "Emission Proability","Transit Probability"]
default_attribute = "error"
Y = np.random.rand(N)
data_point = Scatter(x=X, y=Y, scales={'x': sc_x, 'y': sc_y},
                     default_size=200,colors=["orange"],hovered_style={             
    "fill":"white",
    "stroke":"#9932CC",
    "stroke-width":5
})
# data_point = Scatter(x=X, y=details[default_attribute].values, scales={'x': sc_x, 'y': sc_y},
#                      default_size=200,colors=["orange"])
dropdown_options = []
for d,v in zip(descriptions,attributes):
    dropdown_options.append((d,v))
  
data_dropdown = Dropdown(
    options=dropdown_options,
    value=default_attribute,
    description='Feature:',
    layout=Layout(
         width="50%"   
    )
)

data_label = Label()

ax_x = Axis(scale=sc_x, label='Point index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='-')    
fig = Figure(marks=[data_point], axes=[ax_x, ax_y])

In [34]:
hl = Layout(flex="l",width="50%")
HBox([m,VBox([HBox([data_dropdown,data_label]),fig],layout=hl)])

SEJveChjaGlsZHJlbj0oTWFwKGNlbnRlcj1bNTkuMzQ0NjEzNDMwODYwMzIsIDE4LjA0ODY5MTc0OTU3Mjc1N10sIGNvbnRyb2xzPShab29tQ29udHJvbChvcHRpb25zPVt1J3Bvc2l0aW9uJyzigKY=


In [8]:
def callback(func):
    return func(5)

class Foo:
    def __init__(self,name):
        self.name = name
    def print_name(self, arg):
        print(self.name+" "+str(arg))

foo = Foo("Aria")
callback(foo.print_name)

Aria 5
